In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Chatbot.pdf")
docs = loader.load()
docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

In [ ]:
documents=text_splitter.split_documents(docs)
documents

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents,embedding=OllamaEmbeddings(model='llama3'))

In [ ]:
db

In [ ]:
query="what is Langchain?"
result=db.similarity_search(query)
result[0].page_content

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.llms.llamacpp import LlamaCpp

## Load Ollama LlAMA3 LLM model
llm=Ollama(model="llama3")

llm

In [ ]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
Act like you are directly answering the user's question without mentioning words like "Based on the context".
<context>
{context}
</context>
Question: {input}""")

In [ ]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:

#https://python.langchain.com/docs/modules/data_connection/retrievers/   


retriever=db.as_retriever()
retriever

In [ ]:

#https://python.langchain.com/docs/modules/chains/

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"What is Langchain"})

In [ ]:
response['answer']